If any model error arises, restart the kernel

In [1]:
pip install -q bitsandbytes trl 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from collections import Counter

# Load the dataset
dataset = load_dataset("nvidia/HelpSteer2", split="train")

# ---- Step 1: Remove samples where helpfulness == 3 ----
dataset = dataset.filter(lambda x: x["helpfulness"] != 3)

# ---- Step 2: Create binary labels with >3 ----
dataset = dataset.map(lambda x: {"label": x["helpfulness"] > 3})

# ---- Step 3: Prepare completion field ----
dataset = dataset.map(
    lambda x: {"completion": x["response"]},
    remove_columns=["response"]
)

# ---- Step 4: Compute length of each example (prompt + completion) ----
def add_len(example):
    prompt_text = example.get("prompt", "") or example.get("completion", "")  # Handle different possible field names
    completion_text = example["completion"]
    example["total_length"] = len(prompt_text) + len(completion_text)
    return example

dataset = dataset.map(add_len)

# ---- Step 5: Split dataset by label ----
true_set = dataset.filter(lambda x: x["label"] == True)
false_set = dataset.filter(lambda x: x["label"] == False)

print(f"Before balancing: True={len(true_set)}, False={len(false_set)}")

# ---- Step 6: Balance counts ----
min_count = min(len(true_set), len(false_set))

# ---- Step 7: Sort each set by length (largest first) ----
true_sorted = true_set.sort("total_length", reverse=True)
false_sorted = false_set.sort("total_length", reverse=True)

# ---- Step 8: Trim each set by keeping only the top min_count largest examples ----
true_balanced = true_sorted.select(range(min_count))  # Keep first min_count examples (largest)
false_balanced = false_sorted.select(range(min_count))  # Keep first min_count examples (largest)

# ---- Step 9: Combine ----
from datasets import concatenate_datasets
balanced_dataset = concatenate_datasets([true_balanced, false_balanced])

print("Final balanced stats:", Counter(balanced_dataset["label"]))
print(f"Final dataset size: {len(balanced_dataset)}")
dataset = balanced_dataset

Before balancing: True=8434, False=6013
Final balanced stats: Counter({True: 6013, False: 6013})
Final dataset size: 12026


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import KTOConfig, KTOTrainer

model_name = "Qwen/Qwen3-0.6B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",   # or torch.bfloat16
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # or ["q_proj","k_proj","v_proj","o_proj"]
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

2025-11-19 17:20:22.625118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763572822.648330     199 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763572822.655193     199 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [4]:

from trl import KTOConfig, KTOTrainer

kto_config = KTOConfig(
    report_to="none",
    output_dir="kto-out",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    # max_length=512,
    # max_prompt_length=384,
    # key flag:
    precompute_ref_log_probs=True,
    bf16=True,
    gradient_checkpointing=True,
)

trainer = KTOTrainer(
    model=model,                 # LoRA / QLoRA model
    # ref_model=None,             # KTO will create / handle ref internally for precompute
    args=kto_config,
    processing_class=tokenizer,
    train_dataset=dataset,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/trl/trainer/kto_trainer.py:364: UserWarning: This trainer will soon be moved to trl.experimental and is a candidate for removal. If you rely on it and want it to remain, please share your comments here: https://github.com/huggingface/trl/issues/4223. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  warnings.warn(


Extracting prompt from train dataset:   0%|          | 0/12026 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/12026 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/12026 [00:00<?, ? examples/s]

Processing tokenized train dataset:   0%|          | 0/12026 [00:00<?, ? examples/s]

...

Train dataset reference log probs:   0%|          | 0/6013 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.504800
20,0.503700
30,0.494300
40,0.494100
50,0.488900
60,0.498400
70,0.493400
80,0.496300
90,0.495700
100,0.499000


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=2256, training_loss=0.5031756252473127, metrics={'train_runtime': 30704.2814, 'train_samples_per_second': 1.175, 'train_steps_per_second': 0.073, 'total_flos': 0.0, 'train_loss': 0.5031756252473127, 'epoch': 3.0})

The dataset is imbalanced. To compensate we change undesirable_weight parameter to 2. Read more here- https://huggingface.co/docs/trl/main/en/kto_trainer#imbalanced-data

In [5]:
trainer.save_model("Qwen3-0.6B-KTO")
tokenizer.save_pretrained("Qwen3-0.6B-KTO")


('Qwen3-0.6B-KTO/tokenizer_config.json',
 'Qwen3-0.6B-KTO/special_tokens_map.json',
 'Qwen3-0.6B-KTO/chat_template.jinja',
 'Qwen3-0.6B-KTO/vocab.json',
 'Qwen3-0.6B-KTO/merges.txt',
 'Qwen3-0.6B-KTO/added_tokens.json',
 'Qwen3-0.6B-KTO/tokenizer.json')

In [7]:
repo_id = "AIPlans/Qwen3-0.6B-KTO"

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/AIPlans/Qwen3-0.6B-KTO/commit/02bceae8ae07f474841d264a69de6175148b1721', commit_message='Upload tokenizer', commit_description='', oid='02bceae8ae07f474841d264a69de6175148b1721', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AIPlans/Qwen3-0.6B-KTO', endpoint='https://huggingface.co', repo_type='model', repo_id='AIPlans/Qwen3-0.6B-KTO'), pr_revision=None, pr_num=None)